# BuildSelectionDataSet-NEW-ON-EVERYDATA
 - Making selection dataset from incremental results

In [1]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os
from copy import deepcopy
import collections

## NQ TEST DPR 

In [2]:
# input_file = f'/data/philhoon-relevance/FiD/results/NQ_DPR/TEST/incremental_result_{sample_size}/ctx{sample_size}.json'
# output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_TEST_DPR_SELECTION_NEW/'

## NQ DEV DPR 

In [3]:
dataset = 'NQ'
retriever = 'DPR'
datatype = 'DEV'
sample_size = 100

## NQ DEV SEAL

In [ ]:
# dataset = 'NQ'
# retriever = 'SEAL'
# datatype = 'DEV'
# sample_size = 100

## NQ DEV Contriever

In [ ]:
# dataset = 'NQ'
# retriever = 'CONTRIEVER'
# datatype = 'DEV'
# sample_size = 100

## TQA TEST DPR 

In [ ]:
# dataset = 'TQA'
# retriever = 'DPR'
# datatype = 'TEST'
# sample_size = 100

## TQA DEV DPR 

In [ ]:
# dataset = 'TQA'
# retriever = 'DPR'
# datatype = 'DEV'
# sample_size = 100

## TQA DEV CONTRIEVER 

In [ ]:
# dataset = 'TQA'
# retriever = 'CONTRIEVER'
# datatype = 'DEV'
# sample_size = 100

## TQA DEV SEAL 

In [ ]:
# dataset = 'TQA'
# retriever = 'SEAL'
# datatype = 'DEV'
# sample_size = 100

In [4]:
input_file = f'/data/philhoon-relevance/FiD/results/{dataset}_{retriever}/{datatype}/incremental_result_{sample_size}/ctx{sample_size}.json'
print(input_file)

/data/philhoon-relevance/FiD/results/NQ_DPR/DEV/incremental_result_100/ctx100.json


In [5]:
output_path = f'/data/philhoon-relevance/FiD/open_domain_data/{dataset}_{datatype}_{retriever}_SELECTION_NEW/'
print(output_path)

/data/philhoon-relevance/FiD/open_domain_data/NQ_DEV_DPR_SELECTION_NEW/


In [ ]:
print(input_file)
print(output_path)

In [ ]:
input_ = utils.open_json(input_file)

In [ ]:
def get_definite_pos_neg(test_em):
    positive_pos = []
    if test_em.startswith('1'):
        positive_pos.append(0)
    iter_ = re.finditer(r'01', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        positive_pos.append(pos_)

    negative_pos = []
    iter_ = re.finditer(r'10', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        negative_pos.append(pos_)
    
    return positive_pos, negative_pos

In [ ]:
def get_semi_pos(test_em):
    semi_pos = []
    iter_ = re.finditer(r'(?=(11))', test_em)
    for m in iter_:
        semi_pos_ = m.start() + 1
        semi_pos.append(semi_pos_)
    
    return semi_pos

def get_semi_neg(test_em, num_undecisive):
    semi_neg = []
    test_em_temp = test_em[num_undecisive:]
    iter_ = re.finditer(r'(?=(00))', test_em_temp)
    for m in iter_:
        semi_neg_ = m.start() + 1
        semi_neg.append(semi_neg_)
    semi_neg = [i + num_undecisive for i in semi_neg]
    return semi_neg

In [ ]:
input_[0].keys()

In [ ]:
input_file = input_
method = 'method1'

In [ ]:
output_format = []

for id_, instance in enumerate(input_file, 1):
    template_dict = {}
    if 'id' in instance.keys():
        template_dict['id'] = instance['id']
    else:
        template_dict['id'] = str(id_)
    template_dict['answers'] = instance['answers']
    template_dict['question'] = instance['question']
    template_dict['em_pattern'] = instance['em_pattern']

    em_pattern = deepcopy(instance['em_pattern'])
    instances_ = deepcopy(instance['ctxs'])
    num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))


    ## Meaing no EM
    if num_undecisive == 100:
        template_dict['ctxs']= instance['ctxs']
        output_format.append(template_dict)
    ## Meaning EM
    else:
#          check_lst = []
        # initial zeros
        new_context = []
        initial_zeros_lst = [i for i in range(0, num_undecisive)]

        # defnitie positive, definite negative
        defnite_positve_lst, defnite_negative_lst = get_definite_pos_neg(em_pattern)

        # semi positive, semi negative
        semi_positive_lst = get_semi_pos(em_pattern)
        semi_negative_lst = get_semi_neg(em_pattern, num_undecisive)

        if method == 'method1':
            new_context.extend(defnite_positve_lst)
            new_context.extend(semi_positive_lst)

        elif method == 'method2':
            new_context.extend(defnite_positve_lst)

        elif method == 'method3':
            new_context.extend(initial_zeros_lst)
            new_context.extend(defnite_positve_lst)
            new_context.extend(semi_positive_lst)

        elif method == 'method4':
            new_context.extend(initial_zeros_lst)
            new_context.extend(defnite_positve_lst)

        elif method == 'method5':
            new_context.extend(initial_zeros_lst)
            new_context.extend(defnite_positve_lst)
            new_context.extend(semi_positive_lst)
            new_context.extend(semi_negative_lst)

        elif method == 'method6':
            new_context.extend(initial_zeros_lst)
            new_context.extend(defnite_positve_lst)
            new_context.extend(semi_negative_lst)
        
        
        if defnite_positve_lst and defnite_negative_lst:
            print(f"em_pattern : {instance['em_pattern']}")
            print(f"answer : {instance['question']}")
            print(f"answer : {instance['answers']}")
            print(f"defnite_positve_lst : {defnite_positve_lst}")
            pprint([instance['inference'][i]for i in defnite_positve_lst])
            pprint([instances_[i]for i in defnite_positve_lst])
            print(f"defnite_negative_lst : {defnite_negative_lst}")
            pprint([instance['inference'][i]for i in defnite_negative_lst])
            pprint([instances_[i]for i in defnite_negative_lst])
            
           
            
#         new_context.sort()
#         if [item for item, count in collections.Counter(new_context).items() if count > 1]:
#             print('Duplicates index!')
#         else:
#             new_context_lst = [instances_[i]for i in new_context]
        
        
        
#         template_dict['ctxs'] = new_context_lst
#         output_format.append(template_dict)


In [ ]:
def build_data_2(input_file, method):
    output_format = []

    for id_, instance in enumerate(input_file, 1):
        template_dict = {}
        if 'id' in instance.keys():
            template_dict['id'] = instance['id']
        else:
            template_dict['id'] = str(id_)
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
        template_dict['em_pattern'] = instance['em_pattern']

        em_pattern = deepcopy(instance['em_pattern'])
        instances_ = deepcopy(instance['ctxs'])
        num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))

        
        ## Meaing no EM
        if num_undecisive == 100:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
        ## Meaning EM
        else:
#          check_lst = []
            # initial zeros
            new_context = []
            initial_zeros_lst = [i for i in range(0, num_undecisive)]

            # defnitie positive, definite negative
            defnite_positve_lst, defnite_negative_lst = get_definite_pos_neg(em_pattern)

            # semi positive, semi negative
            semi_positive_lst = get_semi_pos(em_pattern)
            semi_negative_lst = get_semi_neg(em_pattern, num_undecisive)

            if method == 'method1':
                new_context.extend(defnite_positve_lst)
                new_context.extend(semi_positive_lst)

            elif method == 'method2':
                new_context.extend(defnite_positve_lst)

            elif method == 'method3':
                new_context.extend(initial_zeros_lst)
                new_context.extend(defnite_positve_lst)
                new_context.extend(semi_positive_lst)

            elif method == 'method4':
                new_context.extend(initial_zeros_lst)
                new_context.extend(defnite_positve_lst)

            elif method == 'method5':
                new_context.extend(initial_zeros_lst)
                new_context.extend(defnite_positve_lst)
                new_context.extend(semi_positive_lst)
                new_context.extend(semi_negative_lst)

            elif method == 'method6':
                new_context.extend(initial_zeros_lst)
                new_context.extend(defnite_positve_lst)
                new_context.extend(semi_negative_lst)

            new_context.sort()
            if [item for item, count in collections.Counter(new_context).items() if count > 1]:
                print('Duplicates index!')
            else:
                new_context_lst = [instances_[i]for i in new_context]

            template_dict['ctxs'] = new_context_lst
            output_format.append(template_dict)
    return output_format

In [ ]:
method_lst = [
    'method1',
    'method2',
    'method3',
    'method4',
    'method5',
    'method6']

In [ ]:
output_path

In [ ]:
check_method = []
for method in method_lst:

    filename = f'{method}.json'
    output_file = os.path.join(output_path, filename)
    print(output_file)
    output_data = build_data_2(input_, method)
    check_method.append(output_data)
    utils.save_json(output_data, output_file)
    print(f'{output_file} saved')
            

## Mtehod1 Verify

In [ ]:
def m1_v(output_format):
    for iins_ in output_format:
        check_counter = collections.Counter(iins_['em_pattern'])
        if check_counter['1'] == 0 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if check_counter['1'] > 1 and check_counter['1'] != len(iins_['ctxs']):
            print('context number not match')
        check_counter = None

In [ ]:
m1_v(check_method[0])

## Mtehod2 Verify

In [ ]:
def m2_v(output_format):
    for iins_ in output_format:
        positive_pos, negative_pos = get_definite_pos_neg(iins_['em_pattern'])
        check_counter = collections.Counter(iins_['em_pattern']) 
        if check_counter['0'] == 100 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if check_counter['0'] != 100 and len(iins_['ctxs']) != len(positive_pos):
            print(positive_pos)
            print(len(iins_['ctxs']))
            print('context number not match')
        check_counter = None

In [ ]:
m2_v(check_method[1])

## Mtehod3 Verify

In [ ]:
def m3_v(output_format):
    for iins_ in output_format:
        positive_pos, negative_pos = get_definite_pos_neg(iins_['em_pattern'])
        semi_pos = get_semi_pos(iins_['em_pattern'])
        num_undecisive = len(iins_['em_pattern']) - len(iins_['em_pattern'].lstrip('0'))
        
        if num_undecisive == 100 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if num_undecisive != 100 and len(iins_['ctxs']) != len(positive_pos) + len(semi_pos) + num_undecisive:
            print(iins_['em_pattern'])
            print(len(positive_pos))
            print(len(semi_pos))
            print(num_undecisive)
            print(len(iins_['ctxs']))
            print('context number not match')

In [ ]:
m3_v(check_method[2])

## Method4 Verify

In [ ]:
def m4_v(output_format):
    for iins_ in output_format:
        positive_pos, negative_pos = get_definite_pos_neg(iins_['em_pattern'])
        num_undecisive = len(iins_['em_pattern']) - len(iins_['em_pattern'].lstrip('0'))
        
        if num_undecisive == 100 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if num_undecisive != 100 and len(iins_['ctxs']) != len(positive_pos) + num_undecisive:
            print(iins_['em_pattern'])
            print(len(positive_pos))
            print(len(semi_pos))
            print(num_undecisive)
            print(len(iins_['ctxs']))
            print('context number not match')

In [ ]:
m4_v(check_method[3])

## Method5 Verify

In [ ]:
def m5_v(output_format):
    for iins_ in output_format:
        positive_pos, negative_pos = get_definite_pos_neg(iins_['em_pattern'])
        semi_pos = get_semi_pos(iins_['em_pattern'])
        num_undecisive = len(iins_['em_pattern']) - len(iins_['em_pattern'].lstrip('0'))
        semi_neg = get_semi_neg(iins_['em_pattern'], num_undecisive)
            
        if num_undecisive == 100 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if num_undecisive != 100 and len(iins_['ctxs']) != len(positive_pos) + len(semi_pos) + len(semi_neg) + num_undecisive:
            print(iins_['em_pattern'])
            print(len(positive_pos))
            print(len(semi_pos))
            print(num_undecisive)
            print(len(iins_['ctxs']))
            print('context number not match')

In [ ]:
m5_v(check_method[4])

## Method6 Verify

In [ ]:
def m6_v(output_format):
    for iins_ in output_format:
        positive_pos, negative_pos = get_definite_pos_neg(iins_['em_pattern'])
#         semi_pos = get_semi_pos(iins_['em_pattern'])
        num_undecisive = len(iins_['em_pattern']) - len(iins_['em_pattern'].lstrip('0'))
        semi_neg = get_semi_neg(iins_['em_pattern'], num_undecisive)
            
        if num_undecisive == 100 and len(iins_['ctxs']) != 100:
            print('context not 100 when no answer')
        if num_undecisive != 100 and len(iins_['ctxs']) != len(positive_pos) + len(semi_neg) + num_undecisive:
            print(iins_['em_pattern'])
            print(len(positive_pos))
            print(len(semi_pos))
            print(num_undecisive)
            print(len(iins_['ctxs']))
            print('context number not match')

In [ ]:
m6_v(check_method[5])

In [ ]:
# output_path

In [ ]:
# for 